# Matrix equation with parameter

First attempt at the grading code. Works well, but cannot be used with Open edX, since edX sympy does not include `solveset`.

In [3]:
import re
import sympy
from sympy import S, solveset
import numpy as np

expected = ["k==0", "k!=0", "None"]

class CustomError(Exception): 
    pass

def double_eq(s):
    t =  re.sub(r"(?<![<>=!])=(?!=)", "==", s)
    return t

def format_sol(s):
    s = s.replace(" ", "")
    if s.lower() == "None".lower():
        return "None"
    if s.lower() == "All".lower():
        return "All"
    s = double_eq(s)
    return s


def ineq_set(s, symb):
    k = sympy.Symbol(symb)
    if s == "Any":
        sol_int = S.Reals
    elif s == "None":
         sol_int = S.EmptySet
    elif "!=" in s:
        n = s.find("!=")
        lhs = s[:n]
        rhs = s[n+2:]
        sol_int = solveset((eval(lhs + ">" + rhs)), domain=S.Reals).union(solveset(eval(lhs+  "<" + rhs), domain=S.Reals))
    elif "==" in s:
        n = s.find("==")
        lhs = s[:n]
        rhs = s[n+2:]
        sol_int = solveset(eval("(" + lhs + ") - (" + rhs + ")"), domain=S.Reals)
    else:
        sol_int = solveset(eval(s), domain=S.Reals)
    return sol_int
                           
                           
def compare_exp(s, t, symb = "k"):                       
    s_int = ineq_set(s, symb)
    t_int = ineq_set(t, symb)
    print(s_int)
    print(t_int)
    return s_int.symmetric_difference(t_int) == S.EmptySet


def check_answer(expect, ans):
    try:
        s = format_sol(expect)
        t = format_sol(ans)
        return compare_exp(s, t, symb = "k")
    except Exception:
        raise CustomError("Incorrect format of the answer.")


In [9]:
s = "2 < k"
t = "2*k > 4"
compare_exp(s, t, symb = "k")

Interval.open(2, oo)
Interval.open(2, oo)


True

Second attempt. Seems to work fine too, but more complicated that the previous version. Works with edX though. At the end using `fq.is_constant()` would be better than trying float conversion, but edX does not recognize it. 

In [14]:
import re
import sympy


class CustomError(Exception): 
    pass

def double_eq(s):
    
    t =  re.sub(r"(?<![<>=!])=(?!=)", "==", s)
    return t

def format_sol(s):
    
    s = s.replace(" ", "")
    if s.lower() == "None".lower():
        return "None"
    elif s.lower() == "Any".lower():
        return "Any"
    else:
        return double_eq(s)

def find_oper(s):
    m = re.search(r"[<>=!]=|[<>]", s)
    if m is None:
        return m
    else:
        return m.group(0)

def split(s):
    
    op = find_oper(s)
    
    if op is  None:
        return None
    
    n = s.find(op)
    m = len(op)
    lhs = s[:n]
    rhs = s[n+m:]
    if "<" in op:
        op = op.replace("<", ">")
        lhs, rhs = rhs, lhs
    
    return (lhs, op, rhs)

def check_answer(expect, ans, symb="k"):
    try:
        k = sympy.Symbol(symb)
        s = format_sol(expect)
        t = format_sol(ans)
        if s in ["Any", "None"]:
            return s == t
        if symb not in t:
            return False
        lhs_s, op_s, rhs_s = split(s)
        lhs_t, op_t, rhs_t = split(t)
        if op_s != op_t:
            return False
        comb_s = eval("(" + lhs_s + ") - (" + rhs_s + ")")
        comb_t = eval("(" + lhs_t + ") - (" + rhs_t + ")")
        print(comb_s)
        print(comb_t)
        q = (sympy.cancel(comb_s/comb_t))
        try:
            fq = float(q)
            if fq <= 0 and ">" in op_s:
                return False
            else:
                return True
        except TypeError:
            return False                             
    except Exception:
        raise CustomError("Incorrect format of the answer.")

In [15]:
expect = "k = 2"
ans = "2*k = 4"
check_answer(expect, ans)

k - 2
2*k - 4


True

In [17]:
code = """

import re
import sympy


class CustomError(Exception): 
    pass

def double_eq(s):
    
    t =  re.sub(r"(?<![<>=!])=(?!=)", "==", s)
    return t

def format_sol(s):
    
    s = s.replace(" ", "")
    if s.lower() == "None".lower():
        return "None"
    elif s.lower() == "Any".lower():
        return "Any"
    else:
        return double_eq(s)

def find_oper(s):
    m = re.search(r"[<>=!]=|[<>]", s)
    if m is None:
        return m
    else:
        return m.group(0)

def split(s):
    
    op = find_oper(s)
    
    if op is  None:
        return None
    
    n = s.find(op)
    m = len(op)
    lhs = s[:n]
    rhs = s[n+m:]
    if "<" in op:
        op = op.replace("<", ">")
        lhs, rhs = rhs, lhs
    
    return (lhs, op, rhs)

def check_answer(expect, ans, symb="k"):
    try:
        k = sympy.Symbol(symb)
        s = format_sol(expect)
        t = format_sol(ans)
        if s in ["Any", "None"]:
            return s == t
        if symb not in t:
            return False
        lhs_s, op_s, rhs_s = split(s)
        lhs_t, op_t, rhs_t = split(t)
        if op_s != op_t:
            return False
        comb_s = eval("(" + lhs_s + ") - (" + rhs_s + ")")
        comb_t = eval("(" + lhs_t + ") - (" + rhs_t + ")")
        print(comb_s, "   ", comb_t)
        q = (sympy.cancel(comb_s/comb_t))
        try:
            fq = float(q)
            if fq <= 0 and ">" in op_s:
                return False
            else:
                return True
        except TypeError:
            return False                             
    except Exception:
        raise CustomError("Incorrect format of the answer.")

"""

In [16]:
def edX(code):
    code = code.replace("<", "&lt;")
    code = code.replace(">", "&gt;")
    return code

In [19]:
print(edX(code))




import re
import sympy


class CustomError(Exception): 
    pass

def double_eq(s):
    
    t =  re.sub(r"(?&lt;![&lt;&gt;=!])=(?!=)", "==", s)
    return t

def format_sol(s):
    
    s = s.replace(" ", "")
    if s.lower() == "None".lower():
        return "None"
    elif s.lower() == "Any".lower():
        return "Any"
    else:
        return double_eq(s)

def find_oper(s):
    m = re.search(r"[&lt;&gt;=!]=|[&lt;&gt;]", s)
    if m is None:
        return m
    else:
        return m.group(0)

def split(s):
    
    op = find_oper(s)
    
    if op is  None:
        return None
    
    n = s.find(op)
    m = len(op)
    lhs = s[:n]
    rhs = s[n+m:]
    if "&lt;" in op:
        op = op.replace("&lt;", "&gt;")
        lhs, rhs = rhs, lhs
    
    return (lhs, op, rhs)

def check_answer(expect, ans, symb="k"):
    try:
        k = sympy.Symbol(symb)
        s = format_sol(expect)
        t = format_sol(ans)
        if s in ["Any", "None"]:
            return s == t
        if s